In [44]:
#This script performs several setps of time series analyis
# First section: Order selection for ARIMAX model based on lowest AIC and BIC for both the base case (without ex. variables)
# and the complete model using sentiment scores
# Second section: Testing for model performance improvement when including sentiment scores
# Third section: Testing for model performance improvement when including sentiment scores and economic variables
# Fourth section: Fitting LSTM model for comparison, again with base case, including only sentiment scores, and including both sentiment scores and economic variables

# The aim of our research is not to identify multicollinearity, endogeneity, the only purpose is to test in all the following scripts whether the predictive power of two different models can be improved.

import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

df = pd.read_csv("./tsdata/modeldata.csv", index_col='Date', parse_dates=True)
df.index.freq = "MS"
ts_returns = pd.Series(df['returns'], index=df.index)
ts_msci = pd.Series(df['msci'], index=df.index)

returntrain = ts_returns[:'2020-01-01']
returntest = ts_returns['2020-01-02':]

mscitrain = ts_msci[:'2020-01-01']
mscitest = ts_msci ['2020-01-01':]

In [45]:
#First section

def evaluateModel(data, exog, order):
    model = SARIMAX(data, exog=exog, order=(order[0], 0, order[1]), enforce_stationarity=True, enforce_invertibility=True)
    model_fit = model.fit(disp=False, maxiter = 300)
    return order, model_fit.aic, model_fit.bic

orders = [(p, q) for p in range(5) for q in range(5)]

results_base = []
results_exog = []

#Base model
for order in orders:
    try:
        order, aic, bic = evaluateModel(returntrain, None, order)
        results_base.append([order, aic, bic])
    except Exception as e:
        print(f"Order {order} failed: {e}")

#Including sentiment scores
for order in orders:
    try:
        order, aic_exo, bic_exo = evaluateModel(returntrain, mscitrain, order)
        results_exog.append([order, aic_exo, bic_exo])
    except Exception as e:
        print(f"Order {order} with exogenous variables failed: {e}")

df_base = pd.DataFrame(results_base, columns=['Order', 'AIC', 'BIC'])
df_exog = pd.DataFrame(results_exog, columns=['Order', 'AIC', 'BIC'])

/Users/MaximilianDroschl/anaconda3/envs/saleseq/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/MaximilianDroschl/anaconda3/envs/saleseq/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/MaximilianDroschl/anaconda3/envs/saleseq/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/MaximilianDroschl/anaconda3/envs/saleseq/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found

In [46]:
df_base

,Order,AIC,BIC
0,"(0, 0)",-269.736521,-267.161810
1,"(0, 1)",-270.748852,-265.599430
2,"(0, 2)",-268.989886,-261.265753
3,"(0, 3)",-267.458180,-257.159336
4,"(0, 4)",-266.762281,-253.888726
5,"(1, 0)",-270.491336,-265.341914
6,"(1, 1)",-269.788873,-262.064740
7,"(1, 2)",-267.878829,-257.579985
8,"(1, 3)",-266.150924,-253.277370
9,"(1, 4)",-265.173835,-249.725569


In [47]:
df_exog

,Order,AIC,BIC
0,"(0, 0)",-268.007367,-262.857945
1,"(0, 1)",-269.000040,-261.275907
2,"(0, 2)",-267.402336,-257.103492
3,"(0, 3)",-265.867584,-252.994030
4,"(0, 4)",-265.156226,-249.707961
5,"(1, 0)",-268.686876,-260.962743
6,"(1, 1)",-268.304697,-258.005853
7,"(1, 2)",-266.336875,-253.463320
8,"(1, 3)",-264.519651,-249.071385
9,"(1, 4)",-263.581810,-245.558834
